<a href="https://colab.research.google.com/github/Uchirgerel/Course-iin-ajil/blob/master/Ug_taamaglalt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Ашигласан сангууд numpy , tensorflow , keras , nltk
import numpy ,tensorflow, keras, nltk
import numpy as np

In [0]:
# Хэрэгтэй сангууд
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [112]:
# Датагаа аван Корпусныхаа хэмжээ(урт)-г авах
path = '/content/data.txt'
text = open(path).read().lower()
print('corpus urt:', len(text))

corpus urt: 11492


In [0]:
# Ямар нэгэн тусгай тэмдэгтгүйгээр датаныхаа үг тус бүрийг салгаж авах
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [114]:
# WORD_LENGTH - нь хэдэн үг урд орсон байна тэр тоогоор дараачийн үгийг
# таамаглана. Үгийн урт нь дээд талдаа 5 байх бөгөөд prev_words гэх хоосон массив
# үүсгэн түүндээ урд буй 5 үгээ хадгалах болно
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['how', 'to', 'fix', 'a', 'catalytic']
converter


In [0]:
# Энд 2 numpy массивийг зарлан X-д нь үгийн онцлог Y-д нь дарааллын үгийг 
# хадгалана. Тухайн үгийн байрлал нь 1-тэй тэнцэх үед X болон Y-ийг давталтанд
# оруулна
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
  for j, each_word in enumerate(each_words):
    X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [116]:
# нэг дарааллын жишээг шалгаж буй нь
print(X[0][0])

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False Fa

In [0]:
# 128 neuron-той бүрэн холбогдсон 1 давхаргатай softmax функц ашиглан model-ээ бэлдэв 
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [127]:
# RMSprop ашиглан 100 epoch явуулав
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=100, shuffle=True).history

Train on 1369 samples, validate on 73 samples
Epoch 1/100
1369/1369 [==============================] - 1s 545us/step - loss: 4.4722 - accuracy: 0.1454 - val_loss: 7.0137 - val_accuracy: 0.0137
Epoch 2/100
1369/1369 [==============================] - 0s 329us/step - loss: 3.4050 - accuracy: 0.2615 - val_loss: 7.4313 - val_accuracy: 0.0000e+00
Epoch 3/100
1369/1369 [==============================] - 0s 328us/step - loss: 2.5847 - accuracy: 0.3791 - val_loss: 8.4491 - val_accuracy: 0.0274
Epoch 4/100
1369/1369 [==============================] - 0s 328us/step - loss: 1.9973 - accuracy: 0.4916 - val_loss: 8.7599 - val_accuracy: 0.0137
Epoch 5/100
1369/1369 [==============================] - 0s 321us/step - loss: 1.4587 - accuracy: 0.6245 - val_loss: 9.6713 - val_accuracy: 0.0274
Epoch 6/100
1369/1369 [==============================] - 0s 353us/step - loss: 1.0005 - accuracy: 0.7538 - val_loss: 10.1585 - val_accuracy: 0.0411
Epoch 7/100
1369/1369 [==============================] - 0s 326us/s

In [0]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))

model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [0]:
# Одоо таамаглах
# Таамаглахдаа жижиг өгөгдөл өгөн түүгээрээ залан вектор болгож өгнө 
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
    
    prepare_input("It is not a lack".lower())

In [0]:
# Моделоо ашиглан таамаглахдаа Sample функцыг ашиглан хамгийн боломжит хувилбарыг олно 
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [0]:
# predict_completions функц моделыг ашиглаж үгсийг таамаглан буцаана 
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [134]:
q =  "How to fix catalytic"
print("зөв өгүүлбэр: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Дараалал: ",seq)
print("Дараагийн үгнүүд: ", predict_completions(seq, 5))

зөв өгүүлбэр:  How to fix catalytic
Дараалал:  how to fix catalytic
how
to
fix
catalytic
Дараагийн үгнүүд:  ['contamination', 'consider', 'in', 'without', 'signs']
